In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
import pickle

In [ ]:
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

In [ ]:
file_path  = '/data/share/project01/gender_age_dataset.txt'

In [ ]:
nrows = None
df = pd.read_csv(file_path, sep='\t', nrows=nrows)

In [ ]:
df['user_json'] = df['user_json'].apply(lambda x: json.loads(x)['visits'])

In [ ]:
# отделяем данные для обучения
df = df.loc[(df['age'] != '-') & (df['gender'] != '-')]

In [ ]:
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote

def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    if netloc is not None: return netloc.strip()
    return None

In [ ]:
# добавляем столбец со списком доменов
df['domains'] = df['user_json'].progress_apply(lambda x: [url2domain(el['url']) for el in x])

In [ ]:
# set из доменов
domains_set = list(set([y for x in df['domains'].tolist() for y in x]))

In [ ]:
len(domains_set)

In [ ]:
import requests
from requests.exceptions import ConnectionError, HTTPError, Timeout

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
def get_domain_text(domain_url):
#     print(domain)
    try:
        domain_txt_src = requests.get('http://' + domain_url, timeout=10).text.lower()
        soup = BeautifulSoup(domain_txt_src, 'lxml')
        for script in soup(["script", "style"]):
            script.decompose()
        domain_txt = ' '.join(soup.get_text().split())
        return domain_txt
    except Exception as e:
        print(e)

In [ ]:
from multiprocessing.dummy import Pool

In [ ]:
p = Pool(32)

In [ ]:
import pickle

In [ ]:
f = open('domain_texts3.pickle', 'wb')

In [ ]:
domains_set = domains_set[78000:]

In [ ]:
for domain, text in tqdm_notebook(zip(domains_set, p.imap(get_domain_text, domains_set))):
    pickle.dump((domain, text), f)